In [8]:
# Pricing American-Style Chooser Options with the Longstaff-Schwartz Method

## 1. Import Required Libraries
#First, we need to import the Python libraries required for the program, including `numpy` for numerical computations and `LinearRegression` from `sklearn.linear_model` for linear regression analysis.
#```python
import numpy as np
from sklearn.linear_model import LinearRegression

In [9]:
# Basic parameter settings
S0 = 100  # Initial stock price
K = 100   # Strike price
r = 0.05  # Risk-free interest rate
sigma = 0.2  # Stock price volatility
T = 1.0   # Time to option maturity (in years)
U = 0.5   # Choice time for the chooser option (in years)
n_steps = 50  # Number of time steps
n_paths = 10000  # Number of Monte Carlo simulation paths

# Calculate derived parameters
dt = T / n_steps  # Time interval per step
discount = np.exp(-r * dt)  # Single-step discount factor
U_step = int(U / dt)  # Time step corresponding to the choice time

In [10]:
# Generate random numbers and initialize the stock price array
np.random.seed(42)  # Set random seed to ensure result reproducibility
Z = np.random.randn(n_paths, n_steps)  # Random numbers following standard normal distribution
S = np.zeros((n_paths, n_steps + 1))  # Array to store stock price paths
S[:, 0] = S0  # Set initial stock price for all paths

# Simulate stock price paths (Geometric Brownian Motion)
for t in range(1, n_steps + 1):
    S[:, t] = S[:, t - 1] * np.exp(
        (r - 0.5 * sigma **2) * dt + sigma * np.sqrt(dt) * Z[:, t - 1]
    )

In [11]:
def lsm_option_value(S_paths, K, r, dt, option_type='call'):
    """
    Calculate the value of American options using the Longstaff-Schwartz method.
    
    Parameters:
    S_paths: Array of stock price paths, with shape (n_paths, n_steps+1)
    K: Strike price of the option
    r: Risk-free interest rate
    dt: Time interval per step
    option_type: Type of option ('call' for call option, 'put' for put option; default is 'call')
    
    Returns:
    Average value of the option across all simulation paths
    """
    n_paths, n_steps = S_paths.shape
    cashflows = np.zeros_like(S_paths)  # Array to store cash flows at each time step for each path
    
    # Initialize cash flows at option maturity
    if option_type == 'call':
        cashflows[:, -1] = np.maximum(S_paths[:, -1] - K, 0)  # Payoff of call option at maturity
    else:
        cashflows[:, -1] = np.maximum(K - S_paths[:, -1], 0)  # Payoff of put option at maturity
    
    # Backward iteration starting from the step before maturity
    for t in range(n_steps - 2, -1, -1):
        # Filter paths where the option is in-the-money (ITM) (exercising may yield value here)
        itm = (cashflows[:, t + 1] > 0)
        X = S_paths[itm, t]  # Stock prices at time t for ITM paths
        Y = cashflows[itm, t + 1] * discount  # Discounted cash flows from the next step for ITM paths
        
        if len(X) == 0:  # Skip if there are no ITM paths
            continue
        
        # Reshape X to 2D array (required by scikit-learn's LinearRegression)
        X = X.reshape(-1, 1)
        # Fit linear regression model to estimate the value of continuing to hold the option
        model = LinearRegression().fit(X, Y)
        continuation = model.predict(X)  # Predicted value of continuing to hold the option
        
        # Calculate the payoff if exercising the option at time t
        exercise = (
            np.maximum(S_paths[itm, t] - K, 0)
            if option_type == 'call'
            else np.maximum(K - S_paths[itm, t], 0)
        )
        
        # Identify paths where exercising is optimal (exercise value > continuation value)
        exercise_indices = np.where(exercise > continuation)[0]
        full_indices = np.where(itm)[0][exercise_indices]
        
        # Update cash flows: record exercise value at current step for optimal exercise paths,
        # and set cash flows for subsequent steps to 0 (since the option is exercised)
        cashflows[full_indices, t] = exercise[exercise_indices]
        cashflows[full_indices, t + 1:] = 0
    
    # Calculate discounted values of cash flows for all paths and take the average
    discounts = np.exp(-r * dt * np.arange(cashflows.shape[1]))
    discounted_values = cashflows * discounts
    return np.mean(np.max(discounted_values, axis=1))

In [12]:
# Extract stock price paths starting from the choice time (U)
S_U = S[:, U_step:]

# Calculate the values of American call and put options after the choice time, respectively
call_val = lsm_option_value(S_U, K, r, dt, option_type='call')
put_val = lsm_option_value(S_U, K, r, dt, option_type='put')

# Calculate the chooser option price: discount the maximum value (call or put) at the choice time
# back to the current time (using the risk-free rate)
chooser_price = np.exp(-r * U) * np.mean(np.maximum(call_val, put_val))

# Print the result
print(f"American-style Chooser Option Price (via LSM): {chooser_price:.4f}")

American-style Chooser Option Price (via LSM): 10.3540
